In [1]:
pip install textblob


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
import json
import os

In [9]:
naivebayes_analyzer1 = NaiveBayesAnalyzer()
naivebayes_analyzer1.train()

In [12]:
topic1_testsent1 = "Last-minute talks fail, massive LAUSD strike and school shutdowns set for Tuesday"
topic2_testsent2 = "Los Angeles schools close as district workers strike over stalled contract negotiations"

#analyze returns the classification first, then positive and negative weights of the sentence
res1 = naivebayes_analyzer1.analyze(topic1_testsent1)
print(res1)
print(res1[0])
res2 = naivebayes_analyzer1.analyze(topic1_testsent1)
print(res2)
print(res2[0])
#BASI

Sentiment(classification='neg', p_pos=0.10473892702084044, p_neg=0.8952610729791588)
neg
Sentiment(classification='neg', p_pos=0.10473892702084044, p_neg=0.8952610729791588)
neg


In [13]:
#BASIL dataset functions
#compare results of data using pattern analyzer compared to correct annotations
naivebayes_main_analyzer = NaiveBayesAnalyzer()
naivebayes_main_analyzer.train()

def naivebayes_analyzer_compare(data, annotations): #compares how many we correctly classify
    num_correct = 0
    total = 0
    sentence_list = []
    for i in data['body-paragraphs']:
        for j in i:
            sentence_list.append(j)
    for phrase in annotations["phrase-level-annotations"]:
        if(phrase["id"] == "title"):#line below is where we input our model
            if (naivebayes_main_analyzer.analyze(data["title"])[0] == phrase["polarity"]):
                num_correct += 1
            total += 1
        else:
            sentence_id = int(phrase["id"][1:])#input model below here too
            if (naivebayes_main_analyzer.analyze(sentence_list[sentence_id])[0] == phrase["polarity"]):
                num_correct += 1
            total += 1
    return total, num_correct

In [14]:
#BASIL dataset analyzing
file_list = []

#open BASIL dataset and gather the articles w/o scores
for i in range(10):
    file_i = os.listdir("BASILdata/articles/201" + str(i))
    for file_name in file_i:
        file = open("BASILdata/articles/201" + str(i) + "/" + file_name, encoding="utf8")
        json_file = json.load(file)
        file_list.append(json_file)
        
annotation_file_list = []

#open BASIL dataset and gather annotations for articles
for i in range(10):
    file_i = os.listdir("BASILdata/annotations/201" + str(i))
    for file_name in file_i:
        file = open("BASILdata/annotations/201" + str(i) + "/" + file_name, encoding="utf8")
        json_file = json.load(file)
        annotation_file_list.append(json_file)

total_correct = 0
total = 0

#compare correct annotations to the results from our model
for i in range(len(file_list)):
    data = file_list[i]
    annotations = annotation_file_list[i]
    a, b = naivebayes_analyzer_compare(data, annotations)
    total += a
    total_correct += b
    
accuracy = total_correct/total
print("Pattern sentiment on BASIL correct: %.3f , BASIL total: %.3f" %(total_correct, total))
print("Pattern sentiment on BASIL ACCURACY: %.4f" % accuracy)

Pattern sentiment on BASIL correct: 545.000 , BASIL total: 1726.000
Pattern sentiment on BASIL ACCURACY: 0.3158
